In [171]:
import numpy as np
import scipy as sp

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import path

import torch
from torch.utils import data
from torch import nn
import torch.nn.functional as F
from torch.optim import SGD

In [172]:
class SynteticSet(data.Dataset):
    def __init__(self, df, features, target="target"):
        target_dict = {"Class_1": 0, "Class_2": 1, "Class_3": 2, "Class_4": 3}
        
        for i in target_dict:
            df[target][df[target] == i] = target_dict[i]
           
        prefix = "feature_{}"
        
        for i,feature in enumerate(features):
            features[i] = prefix.format(str(feature))
        
            
        self.X = torch.tensor(df[features].to_numpy()).float()
        self.Y = torch.tensor(df[target].to_numpy(dtype=np.int64))
        
        
        
    def __len__(self):
        return self.Y.shape[0]
    
    def __getitem__(self, i):
        return self.X[i], self.Y[i]
    

In [173]:
data_path = path.Path(".\\data")
df = pd.read_csv(data_path/"train.csv")
    
dataset = SynteticSet(df, [i for i in range(50)])
dataset_len = len(dataset)

trainset, evalset = data.random_split(dataset, 
                                      [int(dataset_len*0.7), int(dataset_len*0.3)])

trainloader = data.DataLoader(trainset, batch_size=128, shuffle=True)
evalloader = data.DataLoader(evalset, batch_size=128)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [174]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
                nn.Linear(50,4),
                nn.Softmax(dim=1)
            )
    def forward(self, x):
        return self.net(x)

In [175]:
net = NN()
lossF = nn.CrossEntropyLoss()

opt = SGD(net.parameters() ,lr=0.03)

In [176]:
def train_step():
    loss_list = list()
    for x,y in trainloader:

        opt.zero_grad()
        result = net(x)
        loss = lossF(result, y)

        loss.backward()
        opt.step()

        loss_list.append(loss.item())

    print(np.array(loss_list).mean())

In [177]:
for _ in range(100):
    train_step()

1.1864185775655716
1.1724845391086949
1.1711962151135125
1.1706126762700473
1.1702739403496276
1.1700367334773816
1.1698684208571801
1.1697379520649884
1.1696465431663192
1.169572962703391
1.1694952735517317
1.169459666586881
1.1694219764015574
1.1693691546999776
1.1693321469714917
1.1693049126612858
1.1692762845612967
1.169243421153569
1.169235057761072
1.1692045432340075
1.1691774594064603
1.169164233713307
1.1691559413669315
1.1691354687514646
1.1691038410232097
1.1691267008755282
1.169106463191714
1.1691113285434311
1.1690593519855896
1.1690733027414604
1.1690680527381967
1.1690636236881207
1.1690361347233356
1.1690323492290768
1.169016893428684
1.1690223319438955
1.1690217250008035
1.1690397704977005
1.1690034447901863
1.1690098871042784
1.168990465361196
1.1689771538024847
1.168988569998872
1.1689793187477906
1.168957561200018
1.1689502231817577
1.1689745174683426
1.1689627706677648
1.1689549741187086
1.1689504021046584
1.168971385336881
1.1689541361667557
1.1689399753235812
1.16

In [143]:
x, y = evalset[:]
result = net(x[:, 0:4])
accuracy = (result.argmax(dim=1) == y).float().mean()
loss = lossF(result, y)

print(f"Loss: {loss}  Accuracy: {accuracy}")

Loss: 1.17243492603302  Accuracy: 0.571566641330719
